<a href="https://colab.research.google.com/github/iamchenchu/Deep-Learnig-with-TensorFlow/blob/main/05_transfer_learning_in_tensorflow_part1_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning with TensorFlow Part 1 : Feature Extraction

Transfer Learning is leveraging a working model's existing architecture and learned patterns for our own problem.
**There are two main benefits :**

1. Can leverage an existing neural network architechture proven to work on probelms similar to our own.
2. Can leverage a working neural network architecture which has already learned patterns on similar data to our own, then we can adapt those patterns to our own data.


In [36]:
# Are we using a GPU

!nvidia-smi

Tue Dec 19 17:06:21 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0              41W / 300W |    820MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Downloading and becoming one with the data


In [37]:
# Get data (10% of 10 food classes from food101)

import zipfile

#download the data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

#unzip the downloaded file
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip", "r")
zip_ref.extractall()
zip_ref.close()


--2023-12-19 17:06:22--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.18.207, 142.250.145.207, 74.125.128.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.18.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip.1’

10_food_classes_10_ 100%[===================>] 160.74M  35.4MB/s    in 4.5s    

2023-12-19 17:06:26 (35.6 MB/s) - ‘10_food_classes_10_percent.zip.1’ saved [168546183/168546183]



In [38]:
# How many images in each folder ?
import os

#walk through 10 percent data directory and list number of files
for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")


There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/train'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_curry'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/sushi'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/pizza'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ice_cream'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ramen'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/fried_rice'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/steak'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/grilled_salmon'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/hamburger'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/c

# Creating data loaders (Preparing the data)

We'll use the ImageDataGenerator class to load in our images in batches.

In [39]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

train_dir = "10_food_classes_10_percent/train"
test_dir = "10_food_classes_10_percent/test"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Training images")
train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                                          target_size = IMAGE_SHAPE,
                                                          batch_size = BATCH_SIZE,
                                                          class_mode = "categorical")
print("Testing images :")
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size = IMAGE_SHAPE,
                                             batch_size = BATCH_SIZE,
                                             class_mode = "categorical")



Training images
Found 750 images belonging to 10 classes.
Testing images :
Found 2500 images belonging to 10 classes.


# Setting up callbacks (things to run whilst our model trains)

Callbacks are extra functionality you can ass to your models to be performed during  or after training. Some of the the most popular callbacks :

Call backs are a tool whichh can add helpful functionality to your models during training evaluation or interference

* Tracking experiments with the TensorBoard Callback
* Model checkpoint with the modelCheckpoint callback
* Stopping a model from training (Before it trains too long and overfit) with the earlyStopping callback.





In [40]:
# Create TensorBoard callback (functionized because we need to create a new one for each model)

import datetime

def create_tensoreboard_callback(dir_name, experiment_name) :
  log_dir = dir_name + "/ " + experiment_name + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
  print(f"Saving TensorBoard log files to : {log_dir}")
  return tensorboard_callback



**Note :** You can customize the directory where your TensorBoard logs (model training metrics) get saved to whatever you like. The log_dir parameter we've created above is only one option.

# Creating models using the tensorflow Hub

In the past we've used TensorFlow to create our own models layer by layer from the scratch.

Now we're going to do a similar process, except one majority of your models layers are going to come from TensorFlow Hub.

We can access pretrained models on : https://tfhub.dev/

Browsing the tensorflow hub page and sorting for image classification , we found the following feature vector model link:
https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1


**TensorFLow Hub :** A place to find a plethora of pre-trained machine learning models( Ready to be applied and fine-tuned for your own problems)






In [41]:

# Resnet 50 V2 feature vector
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

# Original: EfficientNetB0 feature vector (version 1)
efficientnet_url = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2"

# # New: EfficientNetB0 feature vector (version 2)
# efficientnet_url = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2"

In [42]:
#Import dependencies

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers


In [55]:
def create_model(model_url, num_classes = 10):
  """ Takes a TensorFlow Hub URL and creates a keras Sequential model with it.
Args : model_url(str): A TensorFlow Hub feature extraction URL.
num_classes(int): The number of output neurons in the output layer, should be equal to the number of target classes, default 10
Returns: An uncompiled keras Sequential model with model_url as a feature extractor layer and a dense output layer with num_classes output neurons.
"""

  # Download the pre-trained model and save it as a Keras layer.
  feature_extractor_layer = hub.KerasLayer(model_url,
                                         trainable = False,
                                         name = "feature_extractor_layer",
                                         input_shape = IMAGE_SHAPE+(3,))
  #Create our sequential model
  model = tf.keras.Sequential([
      feature_extractor_layer,
      layers.Dense(num_classes, activation = "softmax", name ="output_layer")
  ])
  print("Model created successfully")
  print("Model summary:")

  return model

# Creating and testing ReseNet TensorFLow Hub



In [56]:
resnet_model = create_model(resnet_url,
                            num_classes=train_data_10_percent.num_classes)

Model created successfully
Model summary:


In [57]:
resnet_model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extractor_layer (K  (None, 2048)              23564800  
 erasLayer)                                                      
                                                                 
 output_layer (Dense)        (None, 10)                20490     
                                                                 
Total params: 23585290 (89.97 MB)
Trainable params: 20490 (80.04 KB)
Non-trainable params: 23564800 (89.89 MB)
_________________________________________________________________
